In [ ]:
!pip install early-stopping
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gc
import os
import shutil
import time
import zipfile

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from early_stopping import EarlyStopping
from google.colab import drive
from PIL import Image
from sklearn.metrics import roc_auc_score
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torchvision.models import ResNeXt50_32X4D_Weights, resnext50_32x4d
from tqdm import tqdm as tqdm

In [ ]:
#drive.mount('/content/drive')

In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/UCCD3074/Asm2/cassava-leaf-disease-classification.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [ ]:
############################
#Coded by Leong Wai Yin
############################
df = pd.read_csv('/content/cassava-leaf-disease-classification/train.csv')

#train, val, test split (70:15:15)
df_train = df.sample(frac=0.7, random_state=3074)
val_test = df.loc[~df.index.isin(df_train.index)]
df_test = val_test.sample(frac=0.5, random_state=3074)
df_valid = val_test.loc[~val_test.index.isin(df_test.index)]
print(len(df))
print(len(df_train))
print(len(df_valid))
print(len(df_test))

21397
14978
3209
3210


In [ ]:
############################
#Coded by Leong Wai Yin
############################
def get_transform(mode=0):

  train_transform = [
      transforms.Compose([    #no augmentation
        transforms.Resize(512),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
      ]),
      transforms.Compose([    #slight augmentation with cropping
        transforms.Resize(512),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
      ]),
      transforms.Compose([    #heavy augmentation
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(degrees=(0,180)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
      ])
  ]

  val_transform = [
      transforms.Compose([
        transforms.Resize(512),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
      ]),
      transforms.Compose([
        transforms.Resize(512),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
      ]),
      transforms.Compose([
        transforms.Resize(512),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
      ])
  ]  

  return train_transform[mode], val_transform[mode]

In [ ]:
def get_trainloader(batch_size=32, train_transform=None):

  trainset = CasavaDataset(df_train, '/content/cassava-leaf-disease-classification/train_images', transform=train_transform)
  trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

  return trainloader

In [ ]:
def get_validloader(batch_size=32, val_transform=None):

  validset = CasavaDataset(df_valid, '/content/cassava-leaf-disease-classification/train_images', transform=val_transform)
  validloader = DataLoader(validset, batch_size=batch_size, shuffle=False, num_workers=2)

  return validloader

In [ ]:
def get_testloader(batch_size=32, val_transform=None):

  testset = CasavaDataset(df_test, '/content/cassava-leaf-disease-classification/train_images', transform=val_transform)
  testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

  return testloader

In [ ]:
############################
#Adapted from Lab6B
############################
class CasavaDataset(Dataset):

    def __init__(self, csv, root, transform=None):
        self.csv = csv    #store dataset partition csv
        self.root = root    #image dataset directory
        self.transform = transform    #preprocessing and augmentation method
        self.classes = ['CBB', 'CBSD', 'CGM', 'CMD', 'H']   #list of classes

    def __len__(self):
        return self.csv.shape[0]

    def __getitem__(self, idx):
        
        #get image by index
        row=self.csv.iloc[idx]
        img = os.path.join(self.root, row.image_id)
        image = Image.open(img)
        
        #transformation
        if self.transform is not None:
          image = self.transform(image)
        
        #get image label
        label = row.label
        
        return image, label

In [ ]:
############################
#Adapted from lab6A
############################
def build_network(weights=None, model):
  if model == 'resnext50':
    net = resnext50_32x4d(weights=weights)
    in_c = net.fc.in_features    #get fc input shape
    net.fc = nn.Linear(in_c, 5)   #match model output with number of labels

  if model == 'efficientnetb0':
    net = timm.create_model('tf_efficientnet_b0_ns', pretrained=True)
    in_c = net.classifier.in_features
    net.classifier = nn.Linear(in_c, 5)
  return net

In [ ]:
############################
#Coded by Leong Wai Yin
############################
loss_iter = 1

def train(net, kernel_type, trainloader, validloader, optimizer, scheduler, num_epochs, finetune):
    val_loss_best = 1000
    history = []
    model_file = f'{kernel_type}_{num_epochs}ep_best.pth'

    early_stopping = EarlyStopping(depth=5, ignore=10, method='consistency')    #early stopping to prevent overfitting

    loss_iterations = int(np.ceil(len(trainloader)/loss_iter))
    
    #use gpu if available
    if torch.cuda.is_available(): 
        net = net.cuda()
    
    #train mode
    net.train()  
    
    #iterate training for epochs
    for e in range(num_epochs):    
        print(time.ctime(), 'Epoch:', e+1)

        running_loss = 0.0
        running_count = 0.0

        bar = tqdm(trainloader)

        for (inputs, labels) in bar:
            
            #clear gradient
            optimizer.zero_grad()

            #use cuda if available
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()

            #forward propagation
            outs = net(inputs)

            #cross entropy loss calculation
            loss = F.cross_entropy(outs, labels)

            #backprop
            loss.backward()

            #update weights
            optimizer.step()

            #accumulate loss
            running_loss += loss.item()
            running_count += 1

        train_loss = running_loss / running_count
        running_loss = 0. 
        running_count = 0.
        #calculate train loss per epoch
        bar.set_description('loss: %.5f' % (train_loss))   
        
        #validation epoch
        val_loss, acc = validation(net, validloader)    
        
        #train val epoch summary
        if not finetune:    
          content = time.ctime() + ' ' + f'Epoch {e}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {np.mean(train_loss):.5f}, valid loss: {(val_loss):.5f}, acc: {(acc):.4f}'
          print(content)

          #save model if epoch outperform best val loss
          if val_loss < val_loss_best:    
              print('val_loss_best ({:.6f} --> {:.6f}).  Saving model ...'.format(val_loss_best, val_loss))
              torch.save(net.state_dict(), model_file)
              val_loss_best = val_loss

        history.append([e+1,train_loss, val_loss])

        #update lr value
        scheduler.step(val_loss)   
        
         #stop training if no more improvement
        if early_stopping.check(val_loss):   
            print("Early stopping")
            break

    #save model after final epoch
    if not finetune:      
      torch.save(net.state_dict(), f'{kernel_type}_{num_epochs}ep_model.pth')
    
    return history

In [ ]:
####################
#Coded by Leong Wai Yin
####################
def validation(model, validloader):
    model.eval()     #evaluation mode
    LOGITS = []     #append prediction proba per batch
    TARGETS = []    #append actual image label per batch
    
    running_corrects = 0
    running_count = 0

    #validate by batch
    for (inputs, targets) in tqdm(validloader):
        
        #use gpu if available
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            targets = targets.cuda()
        
        #clear gradient
        with torch.no_grad():
            #forward prop
            outputs = model(inputs)
            LOGITS.append(outputs.detach().cpu())
            TARGETS.append(targets.detach().cpu())
            #get argmax label
            _, predicted = torch.max(outputs, 1)
            #acc calculation
            running_corrects += (predicted.view(-1) == targets).sum().double()
            running_count += len(inputs)

    #val loss calculation
    val_loss = F.cross_entropy(torch.cat(LOGITS), torch.cat(TARGETS)).numpy()

    acc = 100*running_corrects/running_count

    return val_loss, acc

In [ ]:
####################
#Coded by Leong Wai Yin
####################
def run(params, finetune=False):
  init_lr = params['lr']            #initial lr
  batch_size = params['batch']      #batch size
  epochs = params['epochs']         #total epoch
  factor = params['factor']         #lr decay factor
  patience = params['patience']     #how long to wait before reduce lr
  eps = params['eps']               #minimum lr
  pretrained_weight = 'IMAGENET1K_V2'
  model = params['model']
  train_transform, val_transform = get_transform(mode=params['transform'])

  net = build_network(pretrained_weight, model)

  optimizer = optim.Adam(net.parameters(), lr=init_lr)

  scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=factor, patience=patience, eps=eps)  #scheduler to reduce lr based on metric changes

  history = train(net, model, get_trainloader(batch_size, train_transform), get_validloader(batch_size, val_transform), optimizer, scheduler, num_epochs=epochs, finetune=finetune)

  #dump model and clear gpu memory
  net = None
  gc.collect()
  torch.cuda.empty_cache()

  return history

In [ ]:
params = {          #best hyperparameters after finetuning
    "lr": 1e-4,
    "batch": 16,
    "epochs": 20,
    "factor": 0.2,
    "patience": 5,
    "eps": 1e-6,
    "transform": 1
    "model": "resnext50"
}
run(params)

In [ ]:
params = {          #best hyperparameters after finetuning
    "lr": 1e-5,
    "batch": 16,
    "epochs": 20,
    "factor": 0.1,
    "patience": 3,
    "eps": 1e-7,
    "transform": 2
    "model": "efficientnetb0"
}
run(params)

In [ ]:
def Sort(sub_li):   #sort training history to find best epoch
    sub_li.sort(key = lambda x: x[2])
    return sub_li

In [ ]:
####################
#Coded by Leong Wai Yin
####################
def fine_tune(model):    #random search finetuning
  epoch = 5
  model_iteration = 30    #no of hyperparameter combinations
  log = []      #to store metric for each model instance

  #range of hyperparameter values to fine tune
  lr_range = 10**np.random.uniform(np.log10(0.00001), np.log10(0.1), size = model_iteration)    #1e-5 to 1e-1
  batch_range = 2**np.random.randint(4, 6, size = model_iteration)                              #16, 32, 64
  factor_range = 10**np.random.uniform(np.log10 (0.01), np.log10(0.9), size = model_iteration)  #0.01 to 0.9
  patience_range = np.random.randint(1, 5, size = model_iteration)                              #1 to 5
  eps_range = 10**np.random.uniform(np.log10(1e-10), np.log10(1e-3), size = model_iteration)    #1e-10 to 1e-3
  transform_range = np.random.randint(0, 2, size = model_iteration)                             #no aug, light aug, heavy aug

  for i in range(model_iteration):      #repeat training for each hyperparameter combination
    params = {
      "lr": lr_range[i].item(),
      "batch": batch_range[i].item(),
      "epochs": epoch,
      "factor": factor_range[i].item(),
      "patience": patience_range[i].item(),
      "eps": eps_range[i].item(),
      "transform": transform_range[i].item(),
      "model": model
    }
    #get train val loss
    stat = run(params)    
    #only keep best epoch metric and append to main log
    log.append([Sort(stat)[0].append(params)])

  print(Sort(log))

In [ ]:
fine_tune("resnext50")

In [ ]:
#copy saved model weights to drive
path = "/content"
dir_list = os.listdir(path)
files = [os.path.join(path, file) for file in dir_list if (file.startswith('resnext') or file.startswith('log'))]
for i in files:
  shutil.copy(os.path.join('/content/', i),"/content/drive/MyDrive/casava/resnext50")